# Baseline model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from hms.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, DropoutLayer, BatchNormalizationLayer
from hms.errors import CrossEntropyError, CrossEntropySoftmaxError, SumOfSquaredDiffsError, L1Error, L2Error
from hms.models import SingleLayerModel, MultipleLayerModel
from hms.initialisers import UniformInit, GlorotUniformInit, ConstantInit
from hms.learning_rules import GradientDescentLearningRule, AdamLearningRule
from hms.data_providers import HMSDataProvider, HMS300dDataProvider
from hms.optimisers import Optimiser
from hms.penalties import L1Penalty, L2Penalty
import seaborn as sns;

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# Seed a random number generator
seed = 6102016 
rng = np.random.RandomState(seed)

# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = HMS300dDataProvider('train', 'intro', 'Wiki', rng=rng)
valid_data = HMS300dDataProvider('validation', 'intro', 'Wiki', rng=rng)
input_dim, output_dim = 300, 6

## Test Penalty coefficients

In [ ]:
weights_penalty_2 = [L2Penalty(1e-1), L2Penalty(1e-2), L2Penalty(1e-4), L2Penalty(1e-6), L2Penalty(1e-8)]
prediction_L2 = []
motion_data = np.load('data/Wiki/validation_intro.npz')['targets']

batch_size = 100  # number of data points in a batch
init_scale = 0.01  # scale for random parameter initialisation
learning_rate = 0.001  # learning rate for gradient descent
num_epochs = 30  # number of training epochs to perform
stats_interval = 1  # epoch interval between recording and printing stats
hidden_dim = 50

for index, weight_penalty in enumerate(weights_penalty_2):
    rng.seed(seed)
    train_data.reset()
    valid_data.reset()

    train_data.batch_size = batch_size 
    valid_data.batch_size = batch_size
    
    weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
    biases_init = ConstantInit(0.)
    
    
    model = MultipleLayerModel([
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
        ReluLayer(),
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
    ])

    error = SumOfSquaredDiffsError()
    learning_rule = AdamLearningRule(learning_rate=learning_rate)
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    optimiser = Optimiser(
            model, error, learning_rule, train_data, valid_data, data_monitors)

    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
    
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    plt.ylim([0.0305, 0.0973])
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    plt.title("Training error and validation error")
    plt.savefig("{} TV error.pdf".format(weight_penalty))
    
    print("Motion length: ", motion_data.shape[0])
    result, evaluation = optimiser.eval_test_set(valid_data, 'validation')
    prediction_300d_L2 = result[-1]
    print('Validation Error with {}:    '.format(weight_penalty) + str(evaluation['errorvalidation']))
    
    prediction_L2.append(prediction_300d_L2)
    
    for i in range(6):
        f, axarr = plt.subplots(2, sharex=True, figsize=(17,8))
        axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
        axarr[0].set_title('Actual motion data')
        axarr[1].plot(prediction_300d_L2[:10000,i], color = 'cadetblue')
        axarr[1].set_title('Raw prediction using L2 penalty')
        plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
        plt.savefig("{0} ComparisonDim_{1}.pdf".format(weight_penalty, i+1))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction_L2[4].transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("Heatmap Wiki")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

## Test number of hidden dimensions (units)

In [ ]:
hidden_dims = [20, 50, 100, 150, 200]

prediction_hidden_dim = []
motion_data = np.load('data/Wiki/validation_extro.npz')['targets']


batch_size = 100  # number of data points in a batch
init_scale = 0.01  # scale for random parameter initialisation
learning_rate = 0.001  # learning rate for gradient descent
num_epochs = 30  # number of training epochs to perform
stats_interval = 1  # epoch interval between recording and printing stats
weight_penalty =  L2Penalty(1e-5)

for hidden_dim in hidden_dims:
    rng.seed(seed)
    train_data.reset()
    valid_data.reset()

    train_data.batch_size = batch_size 
    valid_data.batch_size = batch_size
    
    weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
    biases_init = ConstantInit(0.)
    
    model = MultipleLayerModel([
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
        ReluLayer(),
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
    ])

    error = SumOfSquaredDiffsError()
    learning_rule = AdamLearningRule(learning_rate=learning_rate)
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    optimiser = Optimiser(
            model, error, learning_rule, train_data, valid_data, data_monitors)

    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
    
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    plt.ylim([0.0305, 0.0973])
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    plt.title("Training error and validation error")
    plt.savefig("{} units TVerror.pdf".format(hidden_dim))
        
    result, evaluation = optimiser.eval_test_set(valid_data, 'validation')
    prediction = result[-1]
    print('Validation Error with {} hidden units:    '.format(hidden_dim) + str(evaluation['errorvalidation']))
    
    prediction_hidden_dim.append(prediction)
    
    for i in range(6):
        f, axarr = plt.subplots(2, sharex=True, figsize=(17,8))
        axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
        axarr[0].set_title('Actual motion data')
        axarr[1].plot(prediction[:10000,i], color = 'cadetblue')
        axarr[1].set_title('Raw prediction with {} hidden units'.format(hidden_dim))
        plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
        plt.savefig("{0} units ComparisonDim_{1}.pdf".format(hidden_dim, i+1))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction_hidden_dim[4].transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("Heatmap Wiki")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

## Test 2 hidden layers

In [ ]:
motion_data = np.load('data/Wiki/validation_extro.npz')['targets']

batch_size = 100  # number of data points in a batch
init_scale = 0.01  # scale for random parameter initialisation
learning_rate = 0.001  # learning rate for gradient descent
num_epochs = 30  # number of training epochs to perform
stats_interval = 1  # epoch interval between recording and printing stats
weight_penalty =  L2Penalty(1e-5)
hidden_dim = 100

rng.seed(seed)
train_data.reset()
valid_data.reset()

train_data.batch_size = batch_size 
valid_data.batch_size = batch_size

weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weight_penalty),
    ReluLayer(),    
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
])

error = SumOfSquaredDiffsError()
learning_rule = AdamLearningRule(learning_rate=learning_rate)
data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors)

stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

fig_1 = plt.figure(figsize=(8, 4))
ax_1 = fig_1.add_subplot(111)
# plt.ylim([0.0305, 0.0973])
for k in ['error(train)', 'error(valid)']:
    ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
              stats[1:, keys[k]], label=k)
ax_1.legend(loc=0)
ax_1.set_xlabel('Epoch number')
plt.title("Training error and validation error")
plt.savefig("100 units 2 layers TVerror.pdf")

result, evaluation = optimiser.eval_test_set(valid_data, 'validation')
prediction = result[-1]
print('Validation Error with 2 hidden layers:    ' + str(evaluation['errorvalidation']))

for i in range(6):
    f, axarr = plt.subplots(2, sharex=True, figsize=(17,8))
    axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
    axarr[0].set_title('Actual motion data')
    axarr[1].plot(prediction[:10000,i], color = 'cadetblue')
    axarr[1].set_title('Raw prediction with {} hidden units'.format(hidden_dim))
    plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
    plt.savefig("{0} units ComparisonDim_{1}.pdf".format(hidden_dim, i+1))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction.transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("Heatmap 2 layers")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

## Test Dropout

In [ ]:
incl_probs = [0.2, 0.5, 0.8]
motion_data = np.load('data/Wiki/validation_extro.npz')['targets']
predictions = []

batch_size = 100  
init_scale = 0.01  
learning_rate = 0.001  
num_epochs = 50 
stats_interval = 1 
weight_penalty = L2Penalty(1e-5)
hidden_dim = 150

for incl_prob in incl_probs:
    rng.seed(seed)
    train_data.reset()
    valid_data.reset()

    train_data.batch_size = batch_size 
    valid_data.batch_size = batch_size

    weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
    biases_init = ConstantInit(0.)

    model = MultipleLayerModel([
        DropoutLayer(rng, incl_prob),
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
        ReluLayer(),
        DropoutLayer(rng, incl_prob),
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
    ])

    error = SumOfSquaredDiffsError()
    learning_rule = AdamLearningRule(learning_rate=learning_rate)
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    optimiser = Optimiser(
            model, error, learning_rule, train_data, valid_data, data_monitors)

    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    # plt.ylim([0.0305, 0.0973])
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    plt.title("Training error and validation error")
    plt.savefig("Dropout {} TVerror.pdf".format(incl_prob))

    result, evaluation = optimiser.eval_test_set(valid_data, 'validation')
    prediction = result[-1]
    predictions.append(prediction)
    print('Validation Error with dropout:    ' + str(evaluation['errorvalidation']))

    for i in range(6):
        f, axarr = plt.subplots(2, sharex=True, figsize=(17,8))
        axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
        axarr[0].set_title('Actual motion data')
        axarr[1].plot(prediction[:10000,i], color = 'cadetblue')
        axarr[1].set_title('Raw prediction with {} hidden units'.format(hidden_dim))
        plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
        plt.savefig("Drop {0} ComparisonDim_{1}.pdf".format(incl_prob, i+1))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction.transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("Dropout Heatmap")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

## Test Batch Norm

In [ ]:
motion_data = np.load('data/Wiki/validation_extro.npz')['targets']
batch_size = 100  
init_scale = 0.01  
learning_rate = 0.001  
num_epochs = 1 
stats_interval = 1 
weight_penalty = L2Penalty(1e-5)
hidden_dim = 150

rng.seed(seed)
train_data.reset()
valid_data.reset()

train_data.batch_size = batch_size 
valid_data.batch_size = batch_size

weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
    BatchNormalizationLayer(input_dim = 150),
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
])

error = SumOfSquaredDiffsError()
learning_rule = AdamLearningRule(learning_rate=learning_rate)
data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors)

stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

fig_1 = plt.figure(figsize=(8, 4))
ax_1 = fig_1.add_subplot(111)
# plt.ylim([0.0305, 0.0973])
for k in ['error(train)', 'error(valid)']:
    ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
              stats[1:, keys[k]], label=k)
ax_1.legend(loc=0)
ax_1.set_xlabel('Epoch number')
plt.title("Training error and validation error")
plt.savefig("BN TVerror.pdf")

result, evaluation = optimiser.eval_test_set(valid_data, 'validation')
prediction = result[-1]
print('Validation Error with dropout:    ' + str(evaluation['errorvalidation']))

for i in range(6):
    f, axarr = plt.subplots(2, sharex=True, figsize=(17,8))
    axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
    axarr[0].set_title('Actual motion data')
    axarr[1].plot(prediction[:10000,i], color = 'cadetblue')
    axarr[1].set_title('Raw prediction with {} hidden units'.format(hidden_dim))
    plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
    plt.savefig("BN ComparisonDim_{0}.pdf".format(i+1))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction.transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("BN Heatmap")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

## Test error/loss function

In [ ]:
motion_data = np.load('data/Wiki/validation_intro.npz')['targets']
batch_size = 100  
init_scale = 0.01  
learning_rate = 0.001  
num_epochs = 10
stats_interval = 1 
weight_penalty = L2Penalty(1e-5)
hidden_dim = 150

rng.seed(seed)
train_data.reset()
valid_data.reset()

train_data.batch_size = batch_size 
valid_data.batch_size = batch_size

weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
])

error = L1Error()
learning_rule = AdamLearningRule(learning_rate=learning_rate)
data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

optimiser_L1 = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors)

stats_L1, keys_L1, run_time_L1 = optimiser_L1.train(num_epochs=num_epochs, stats_interval=stats_interval)



In [ ]:
fig_1 = plt.figure(figsize=(8, 4))
ax_1 = fig_1.add_subplot(111)
# plt.ylim([0.0305, 0.0973])
for k in ['error(train)', 'error(valid)']:
    ax_1.plot(np.arange(1, stats_L1.shape[0]) * stats_interval, 
              stats_L1[1:, keys_L1[k]], label=k)
ax_1.legend(loc=0)
ax_1.set_xlabel('Epoch number')
plt.title("Training error and validation error")
plt.savefig("L1 loss TVerror.pdf")

result, evaluation = optimiser_L1.eval_test_set(valid_data, 'validation')
prediction_L1 = result[-1]
print('Validation Error:    ' + str(evaluation['errorvalidation']))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction_L1.transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("L1 Heatmap")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

In [ ]:
batch_size = 100  
init_scale = 0.01  
learning_rate = 0.001  
num_epochs = 30
stats_interval = 1 
weight_penalty = L2Penalty(1e-5)
hidden_dim = 150

rng.seed(seed)
train_data.reset()
valid_data.reset()

train_data.batch_size = batch_size 
valid_data.batch_size = batch_size

weights_init = GlorotUniformInit(rng=rng, gain=2.**0.5)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weight_penalty),
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weight_penalty),
])

error = SumOfSquaredDiffsError()
learning_rule = AdamLearningRule(learning_rate=learning_rate)
data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

optimiser_L2 = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors)

stats_L2, keys_L2, run_time_L2 = optimiser_L2.train(num_epochs=num_epochs, stats_interval=stats_interval)

fig_1 = plt.figure(figsize=(8, 4))
ax_1 = fig_1.add_subplot(111)
# plt.ylim([0.0305, 0.0973])
for k in ['error(train)', 'error(valid)']:
    ax_1.plot(np.arange(1, stats_L2.shape[0]) * stats_interval, 
              stats_L2[1:, keys_L2[k]], label=k)
ax_1.legend(loc=0)
ax_1.set_xlabel('Epoch number')
plt.title("Training error and validation error")
plt.savefig("L2 loss TVerror.pdf")

result, evaluation = optimiser_L2.eval_test_set(valid_data, 'validation')
prediction_L2 = result[-1]
print('Validation Error with dropout:    ' + str(evaluation['errorvalidation']))

In [ ]:
motion_transpose = np.array(motion_data.transpose())
prediction_transpose = np.array(prediction_L2.transpose())

pearson_correlation_coefficient = np.corrcoef(motion_transpose, prediction_transpose)
plot = sns.heatmap(pearson_correlation_coefficient, center=0, linewidths=.5)
fig = plot.get_figure()
fig.savefig("L2 Heatmap")
for i in range(6):
    print("CC of dimension {}".format(i), " is ", pearson_correlation_coefficient[i,i+6])

In [ ]:
for i in range(6):
    f, axarr = plt.subplots(3, sharex=True, sharey=True, figsize=(15,8))
    axarr[0].plot(motion_data[:10000,i], color = 'cadetblue')
    axarr[0].set_title('Actual motion data')
    axarr[1].plot(prediction_L1[:10000,i], color = 'cadetblue')
    axarr[1].set_title('Raw prediction with L1 at epoch 10'.format(hidden_dim))
    axarr[2].plot(prediction_L2[:10000,i], color = 'cadetblue')
    axarr[2].set_title('Raw prediction with L2 at epoch 30'.format(hidden_dim))    
    plt.suptitle("Comparison of Actual Motion and Prediction on Dimension {}".format(i+1), size=20)
    plt.savefig("loss ComparisonDim_{0}.pdf".format(i+1))

## Define a function that smooths the output

In [ ]:
def smooth_prediction(raw_prediction):
    output_shape = raw_prediction.shape
    # make a matrix that adds 20 lines paddings at the beginning & end of raw prediction
    calculation_frame = np.zeros((output_shape[0]+40, output_shape[1]))
    output = np.zeros(raw_prediction.shape)
    calculation_frame[20:-20, :] = raw_prediction

    for i in range(output_shape[0]):
        output[i,:] = calculation_frame[i+20,:] + \
        0.9 * (calculation_frame[i+21,:] + calculation_frame[i+19,:]) + \
        0.9 * (calculation_frame[i+22,:] + calculation_frame[i+18,:]) + \
        0.8 * (calculation_frame[i+23,:] + calculation_frame[i+17,:]) + \
        0.8 * (calculation_frame[i+24,:] + calculation_frame[i+16,:]) + \
        0.7 * (calculation_frame[i+25,:] + calculation_frame[i+15,:]) + \
        0.7 * (calculation_frame[i+26,:] + calculation_frame[i+14,:]) + \
        0.6 * (calculation_frame[i+27,:] + calculation_frame[i+13,:]) + \
        0.6 * (calculation_frame[i+28,:] + calculation_frame[i+12,:]) + \
        0.5 * (calculation_frame[i+29,:] + calculation_frame[i+11,:]) + \
        0.5 * (calculation_frame[i+30,:] + calculation_frame[i+10,:]) + \
        0.4 * (calculation_frame[i+31,:] + calculation_frame[i+9,:]) + \
        0.4 * (calculation_frame[i+32,:] + calculation_frame[i+8,:]) + \
        0.3 * (calculation_frame[i+33,:] + calculation_frame[i+7,:]) + \
        0.3 * (calculation_frame[i+34,:] + calculation_frame[i+6,:]) + \
        0.2 * (calculation_frame[i+35,:] + calculation_frame[i+5,:]) + \
        0.2 * (calculation_frame[i+36,:] + calculation_frame[i+4,:]) + \
        0.1 * (calculation_frame[i+37,:] + calculation_frame[i+3,:]) + \
        0.1 * (calculation_frame[i+38,:] + calculation_frame[i+2,:]) + \
        0.1 * (calculation_frame[i+39,:] + calculation_frame[i+1,:])
    output /= [2.0, 2.0, 2.0, 3.0, 2.0, 2.0]

    # Adjust motion to origin
#     motion_mean = np.mean(output,axis=0)
#     output = output - motion_mean

    return output

In [ ]:
def dct_smoothing(raw_prediction, frequency_factor, window_size):
    trans = raw_prediction.transpose()
    output = np.zeros(trans.shape)
    
    window_number = (output.shape[1] + window_size - 1) // window_size
    temp_output = np.zeros((trans.shape[0], window_size*window_number))
    
    # Do dct smoothing on every window individually
    for i in range(trans.shape[0]):
        motion = np.zeros(window_size * window_number)
        motion[:trans.shape[1]] = trans[i,:]
        for j in range(window_number):
            temp = np.zeros(window_size)
            temp_freq = np.zeros(window_size)
            temp_freq[:frequency_factor] = dct(motion[j*window_size:(j+1)*window_size], norm="ortho")[:frequency_factor]
            temp_output[i, j*window_size:(j+1)*window_size] = dct(temp_freq, 3, norm="ortho")
            #ith_dim_freq = np.zeros(trans.shape[1])
            #ith_dim_freq[:frequency_factor] = dct(trans[i], norm="ortho")[:frequency_factor]
            #output[i] = dct(ith_dim_freq, 3, norm="ortho")
    
    output[:, :raw_prediction.shape[0]] = temp_output[:, :raw_prediction.shape[0]]  
    output = output.transpose()
    
    smooth_boundary = output.copy()
    
    # Smooth the boundaries
#     for i in range(1, window_number):
#         mid_left = i * window_size-1
#         mid_right = mid_left + 1
#         start = mid_left - 29
#         end = mid_right + 29
        
#         mid_mean = (output[mid_left] + output[mid_right]) / 2.0
        
#         for j in range(31):
#             smooth_boundary[start + j,:] = (1 - j / 30.0) * output[j] + j / 30.0 * mid_mean
#             smooth_boundary[end - j,:] = (1 - j / 30.0) * output[j] + j / 30.0 * mid_mean
    return output

## Produce output of test sets using L1

In [ ]:
# Loop over 1-6 test cases

for i in range(1,7):
    test_data = HMS300dDataProvider('test{0}'.format(i),'extro', 'Wiki' rng=rng)
    result, evaluation = optimiser_L1.eval_test_set(test_data, 'test')
    print('Error:    ' + str(evaluation['errortest']))
    
    time_intervals = np.loadtxt("ExtrovertRawData/Words/{0}".format(i), usecols=range(4, 6), dtype="int")
    prediction = np.zeros((time_intervals[-1,1] + 300, 6))
    counter = 0
    for line in time_intervals:
        for j in range(line[0], line[1]):
            prediction[j] = result[-1][counter]
            counter += 1
    prediction_smooth = smooth_prediction(prediction)

    np.savetxt('Predictions/300dL1/fake_extro_L1_{0}.txt'.format(i), prediction, fmt="%.7f")
    np.savetxt('Predictions/300dL1/fake_extro_L1_smooth_{0}.txt'.format(i), prediction_smooth, fmt="%.7f")
    
    motion_data = np.loadtxt("ExtrovertRawData/Motion/{0}.rov".format(i), skiprows=17, usecols=range(0, 6))
    print("Motion length: ", motion_data.shape[0])

    # Two subplots, the axes array is 1-d
    f, axarr = plt.subplots(3, sharex=True, sharey=True, figsize=(17,8))
    axarr[0].plot(motion_data[:10000,i-1], color = 'cadetblue')
    axarr[0].set_title('Actual motion data')
    axarr[1].plot(prediction[:10000,i-1], color = 'cadetblue')
    axarr[1].set_title('Raw prediction')
    axarr[2].plot(prediction_smooth[:10000,i-1], color = 'cadetblue')
    axarr[2].set_title('Smoothed prediction')
    plt.suptitle('Test case {0}, comparison on dimention {1}'.format(i, i), size = 20)
    plt.savefig('Predictions/300dL1/L1 fake_extro case {0} dim {1}.pdf'.format(i, i))
    plt.show()

## Produce output of test sets using L2

In [ ]:
# Loop over 1-6 test cases

for i in range(1,7):
    test_data = HMS300dDataProvider('test{0}'.format(i), rng=rng)
    result, evaluation = optimiser_L1.eval_test_set(test_data, 'test')
    print('Error:    ' + str(evaluation['errortest']))
    
    time_intervals = np.loadtxt("ExtrovertRawData/Words/{0}".format(i), usecols=range(4, 6), dtype="int")
    prediction = np.zeros((time_intervals[-1,1] + 300, 6))
    counter = 0
    for line in time_intervals:
        for j in range(line[0], line[1]):
            prediction[j] = result[-1][counter]
            counter += 1
    prediction_smooth = smooth_prediction(prediction)

    np.savetxt('Predictions/300dL2/fake_extro_L2_{0}.txt'.format(i), prediction, fmt="%.7f")
    np.savetxt('Predictions/300dL2/fake_extro_L2_smooth_{0}.txt'.format(i), prediction_smooth, fmt="%.7f")
    
    motion_data = np.loadtxt("ExtrovertRawData/Motion/{0}.rov".format(i), skiprows=17, usecols=range(0, 6))
    print("Motion length: ", motion_data.shape[0])

    # Two subplots, the axes array is 1-d
    f, axarr = plt.subplots(3, sharex=True, sharey=True, figsize=(17,8))
    axarr[0].plot(motion_data[:10000,i-1], color = 'cadetblue')
    axarr[0].set_title('Actual motion data')
    axarr[1].plot(prediction[:10000,i-1], color = 'cadetblue')
    axarr[1].set_title('Raw prediction')
    axarr[2].plot(prediction_smooth[:10000,i-1], color = 'cadetblue')
    axarr[2].set_title('Smoothed prediction')
    plt.suptitle('Test case {0}, comparison on dimention {1}'.format(i, i), size = 20)
    plt.savefig('Predictions/300dL2/L2 fake_extro case {0} dim {1}.pdf'.format(i, i))
    plt.show()